In [ ]:
# check hardware for mic
p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    info = p.get_device_info_by_index(i)
    if info["maxInputChannels"] > 0:
        print(f"[{i}] {info['name']}")
p.terminate()


This cell runs to make your in background records and you set a break point it records in background and saves files into folder

In [ ]:

#number is just to help track files start at 0
y = 0
SAMPLE_RATE = 16000
DURATION_SECONDS = 1
NUM_SAMPLES = int(SAMPLE_RATE * DURATION_SECONDS)
DEVICE_INDEX = 0  # change to your mic index

def record_clip(x):
    save_path="test_recording"+str(x)+"_negative_background.wav"
    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16,
                    channels=1,
                    rate=SAMPLE_RATE,
                    input=True,
                    input_device_index=DEVICE_INDEX,
                    frames_per_buffer=1024)  # smaller buffer = more reliable

    print("🎙️  Recording 1 s sample...")

    frames = []
    for _ in range(int(SAMPLE_RATE / 1024 * DURATION_SECONDS)):
        data = stream.read(1024, exception_on_overflow=False)
        frames.append(data)

    stream.stop_stream()
    stream.close()
    p.terminate()

    # Join frames & convert to float32 waveform
    audio_bytes = b"".join(frames)
    waveform = np.frombuffer(audio_bytes, dtype=np.int16).astype(np.float32) / 32768.0

    # Fix length to NUM_SAMPLES
    waveform = waveform[:NUM_SAMPLES]
    if len(waveform) < NUM_SAMPLES:
        waveform = np.pad(waveform, (0, NUM_SAMPLES - len(waveform)))

    # Save to WAV file
    sf.write(save_path, waveform, SAMPLE_RATE)
    print(f"💾 Saved recording to: {save_path}")
   

    return waveform
#us while loop to get background noise and put a break point when y gets to desried number
# Run it
while True:
    y+= 1 
    record_clip(y)
    if(y >= 7500):
        break
    #record one clip at time and save it

